In [1]:
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import keras
import csv
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


### Load Raw Training Data

In [23]:
inpData = []
with open('usageData.csv') as csv_file:
    lines = csv.reader(csv_file, delimiter=',')
    for row in lines:
        inpData.append(float(row[0]))     

### Set the Target Labels

In [24]:
minThreshold = 30
maxThreshold = 70
target = []
index = 0
while index + 1 < len(inpData):
    if inpData[index + 1] > maxThreshold:# Scale Up
        target.append(1)
    elif inpData[index + 1] < minThreshold: # Scale Down
        target.append(-1)
    else:
        target.append(0)  # Remain the same, i.e. doing nothing
    index += 1

### Preprocessing Input Data

In [25]:
cpuUsage = []
cpuScaling = []
historyParam = 50
index = historyParam

while index<len(inpData):
    currRec = [[inpData[i] / 100] for i in range(index-historyParam,index)]
    cpuUsage.append(currRec)
    index+=1

In [26]:
cpuScaling = target[historyParam-1:len(target)]

cpuUsageData = np.array(cpuUsage,dtype = 'float')
cpuScaling = np.array(cpuScaling,dtype = 'float')

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)

In [27]:
cpuScalingCategorical

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [28]:
cpuUsageData

array([[[0.17216685],
        [0.17216685],
        [0.26593732],
        ...,
        [0.05854671],
        [0.09904096],
        [0.21955456]],

       [[0.17216685],
        [0.26593732],
        [0.15779982],
        ...,
        [0.09904096],
        [0.21955456],
        [0.06350205]],

       [[0.26593732],
        [0.15779982],
        [0.18895963],
        ...,
        [0.21955456],
        [0.06350205],
        [0.06350205]],

       ...,

       [[0.00074294],
        [0.00066903],
        [0.00076267],
        ...,
        [0.19449881],
        [0.19449881],
        [0.2311533 ]],

       [[0.00066903],
        [0.00076267],
        [0.00076267],
        ...,
        [0.19449881],
        [0.2311533 ],
        [0.10099166]],

       [[0.00076267],
        [0.00076267],
        [0.00076216],
        ...,
        [0.2311533 ],
        [0.10099166],
        [0.10099166]]])

In [29]:
newTrainX = cpuUsageData
newTrainY = cpuScalingCategorical
for i in range(1,len(cpuUsageData)):
    scalingVal = np.argmax(cpuScalingCategorical[i])
    if scalingVal == 0:
        for j in range(0,7):
            newTrainX = np.append(newTrainX,np.array([cpuUsageData[i]]),axis = 0)
            newTrainY = np.append(newTrainY,np.array([cpuScalingCategorical[i]]),axis = 0)
        
    if scalingVal == 1:
        for j in range(0,10):
            newTrainX = np.append(newTrainX,np.array([cpuUsageData[i]]),axis = 0)
            newTrainY = np.append(newTrainY,np.array([cpuScalingCategorical[i]]),axis = 0)

X_train, X_test, y_train, y_test = train_test_split(newTrainX, newTrainY, test_size=0.30, random_state=21)

In [30]:
print(len(X_train), len(X_test), len(y_train), len(y_test))
print(cpuScalingCategorical[0])
print(len(cpuScalingCategorical))
print(X_train.shape)

13729 5885 13729 5885
[0. 0. 1.]
4946
(13729, 50, 1)


In [31]:
scalingVal = np.argmax(cpuScalingCategorical[0])
print(cpuScalingCategorical[0])
print(scalingVal)

[0. 0. 1.]
2


In [11]:
# Definition for the agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.95
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Network for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(18, activation='relu'))
        model.add(Dense(12, activation='relu'))
        model.add(Dense(6, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        print(model.summary())
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state, is_train=True):
        if is_train and np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [12]:
# Definition for the environment
class Environment:
    def __init__(self, data, expected):
        self.counter = 0
        self.window = 50
        self.data = data
        self.expected = expected
        
    def reset(self):
        # return the initial state
        state = self.data[0]
        self.counter = 0
        return state
    
    def get_reward(self, action):
        if np.argmax(self.expected[self.counter - 1]) != action:
            return -1
        return 1

    def step(self, action):
        # use the specified action to get the next state
        self.counter += 1
        done = False
        next_state = self.data[self.counter]
        reward = self.get_reward(action)
        if (self.counter + 1 == len(self.data)):
            done = True
        return next_state, reward, done

In [13]:
# Settings for training or testing
window = 50
state_size = window
action_size = 3 # 1, 0 and -1

batch_size = 100
EPISODES = 1500

In [14]:
env = Environment(X_train, y_train)
agent = DQNAgent(state_size, action_size)
agent.load("./save/dqn-4layer.h5")
done = False
for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    while not done:
        action = agent.act(state, True)
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
    print("episode: {}/{}, e: {:.2}"
                  .format(e, EPISODES, agent.epsilon))
    if e % 5 == 0:
        agent.save("./save/dqn-4layer.h5")

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 24)                1224      
_________________________________________________________________
dense_2 (Dense)              (None, 18)                450       
_________________________________________________________________
dense_3 (Dense)              (None, 12)                228       
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 78        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 21        
Total params: 2,001
Trainable params: 2,001
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
episode: 0/1500

episode: 231/1500, e: 0.0099
episode: 232/1500, e: 0.0099
episode: 233/1500, e: 0.0099
episode: 234/1500, e: 0.0099
episode: 235/1500, e: 0.0099
episode: 236/1500, e: 0.0099
episode: 237/1500, e: 0.0099
episode: 238/1500, e: 0.0099
episode: 239/1500, e: 0.0099
episode: 240/1500, e: 0.0099
episode: 241/1500, e: 0.0099
episode: 242/1500, e: 0.0099
episode: 243/1500, e: 0.0099
episode: 244/1500, e: 0.0099
episode: 245/1500, e: 0.0099
episode: 246/1500, e: 0.0099
episode: 247/1500, e: 0.0099
episode: 248/1500, e: 0.0099
episode: 249/1500, e: 0.0099
episode: 250/1500, e: 0.0099
episode: 251/1500, e: 0.0099
episode: 252/1500, e: 0.0099
episode: 253/1500, e: 0.0099
episode: 254/1500, e: 0.0099
episode: 255/1500, e: 0.0099
episode: 256/1500, e: 0.0099
episode: 257/1500, e: 0.0099
episode: 258/1500, e: 0.0099
episode: 259/1500, e: 0.0099
episode: 260/1500, e: 0.0099
episode: 261/1500, e: 0.0099
episode: 262/1500, e: 0.0099
episode: 263/1500, e: 0.0099
episode: 264/1500, e: 0.0099
episode: 265/1

episode: 606/1500, e: 0.0099
episode: 607/1500, e: 0.0099
episode: 608/1500, e: 0.0099
episode: 609/1500, e: 0.0099
episode: 610/1500, e: 0.0099
episode: 611/1500, e: 0.0099
episode: 612/1500, e: 0.0099
episode: 613/1500, e: 0.0099
episode: 614/1500, e: 0.0099
episode: 615/1500, e: 0.0099
episode: 616/1500, e: 0.0099
episode: 617/1500, e: 0.0099
episode: 618/1500, e: 0.0099
episode: 619/1500, e: 0.0099
episode: 620/1500, e: 0.0099
episode: 621/1500, e: 0.0099
episode: 622/1500, e: 0.0099
episode: 623/1500, e: 0.0099
episode: 624/1500, e: 0.0099
episode: 625/1500, e: 0.0099
episode: 626/1500, e: 0.0099
episode: 627/1500, e: 0.0099
episode: 628/1500, e: 0.0099
episode: 629/1500, e: 0.0099
episode: 630/1500, e: 0.0099
episode: 631/1500, e: 0.0099
episode: 632/1500, e: 0.0099
episode: 633/1500, e: 0.0099
episode: 634/1500, e: 0.0099
episode: 635/1500, e: 0.0099
episode: 636/1500, e: 0.0099
episode: 637/1500, e: 0.0099
episode: 638/1500, e: 0.0099
episode: 639/1500, e: 0.0099
episode: 640/1

episode: 981/1500, e: 0.0099
episode: 982/1500, e: 0.0099
episode: 983/1500, e: 0.0099
episode: 984/1500, e: 0.0099
episode: 985/1500, e: 0.0099
episode: 986/1500, e: 0.0099
episode: 987/1500, e: 0.0099
episode: 988/1500, e: 0.0099
episode: 989/1500, e: 0.0099
episode: 990/1500, e: 0.0099
episode: 991/1500, e: 0.0099
episode: 992/1500, e: 0.0099
episode: 993/1500, e: 0.0099
episode: 994/1500, e: 0.0099
episode: 995/1500, e: 0.0099
episode: 996/1500, e: 0.0099
episode: 997/1500, e: 0.0099
episode: 998/1500, e: 0.0099
episode: 999/1500, e: 0.0099
episode: 1000/1500, e: 0.0099
episode: 1001/1500, e: 0.0099
episode: 1002/1500, e: 0.0099
episode: 1003/1500, e: 0.0099
episode: 1004/1500, e: 0.0099
episode: 1005/1500, e: 0.0099
episode: 1006/1500, e: 0.0099
episode: 1007/1500, e: 0.0099
episode: 1008/1500, e: 0.0099
episode: 1009/1500, e: 0.0099
episode: 1010/1500, e: 0.0099
episode: 1011/1500, e: 0.0099
episode: 1012/1500, e: 0.0099
episode: 1013/1500, e: 0.0099
episode: 1014/1500, e: 0.0099

episode: 1351/1500, e: 0.0099
episode: 1352/1500, e: 0.0099
episode: 1353/1500, e: 0.0099
episode: 1354/1500, e: 0.0099
episode: 1355/1500, e: 0.0099
episode: 1356/1500, e: 0.0099
episode: 1357/1500, e: 0.0099
episode: 1358/1500, e: 0.0099
episode: 1359/1500, e: 0.0099
episode: 1360/1500, e: 0.0099
episode: 1361/1500, e: 0.0099
episode: 1362/1500, e: 0.0099
episode: 1363/1500, e: 0.0099
episode: 1364/1500, e: 0.0099
episode: 1365/1500, e: 0.0099
episode: 1366/1500, e: 0.0099
episode: 1367/1500, e: 0.0099
episode: 1368/1500, e: 0.0099
episode: 1369/1500, e: 0.0099
episode: 1370/1500, e: 0.0099
episode: 1371/1500, e: 0.0099
episode: 1372/1500, e: 0.0099
episode: 1373/1500, e: 0.0099
episode: 1374/1500, e: 0.0099
episode: 1375/1500, e: 0.0099
episode: 1376/1500, e: 0.0099
episode: 1377/1500, e: 0.0099
episode: 1378/1500, e: 0.0099
episode: 1379/1500, e: 0.0099
episode: 1380/1500, e: 0.0099
episode: 1381/1500, e: 0.0099
episode: 1382/1500, e: 0.0099
episode: 1383/1500, e: 0.0099
episode: 1

In [35]:
# Testing the agent
env = Environment(X_test, y_test)
action_list = []
agent = DQNAgent(state_size, action_size)
agent.load("./save/dqn-4layer30.h5")
done = False
for e in range(1):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    while True:
        action = agent.act(state, False)
        action_list.append(action)
        if done:
            break
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        state = next_state

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 24)                1224      
_________________________________________________________________
dense_27 (Dense)             (None, 18)                450       
_________________________________________________________________
dense_28 (Dense)             (None, 12)                228       
_________________________________________________________________
dense_29 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_30 (Dense)             (None, 3)                 21        
Total params: 2,001
Trainable params: 2,001
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
len(action_list)

5885

In [37]:
print(action_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 2, 0, 0, 0, 1, 0, 2, 1, 1, 0, 0, 0, 1, 0, 2, 0, 0, 1, 1, 1, 2, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 1, 2, 1, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 

In [38]:
print(action_list.count(0))
print(np.argmax(y_test,axis = 1))

4037
[1 1 1 ... 0 1 2]


In [39]:
print(accuracy_score(action_list,np.argmax(y_test,axis = 1)))

0.3610875106202209


In [40]:
y_test

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [41]:
predict_dict = {0:0, 1:1, 2:-1}